In [1]:
import bhc
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from scipy.cluster.hierarchy import dendrogram, linkage

In [2]:
def Bern_gen(nobs, k, theta, seed):
    """Generate Bernoulli distributed data"""
    np.random.seed(seed)
    obs_list = []
    theta_list = (np.repeat(theta,nobs))
    theta_list[:int(nobs/3)] = np.repeat(theta-0.3, int(nobs/3))
    theta_list[-int(nobs/3):] = np.repeat(theta+0.3, int(nobs/3))
    for i in range(nobs):
        X_i = np.random.binomial(1, theta_list[i], k)
        obs_list.append(X_i)
    return np.matrix(obs_list)

X_test = Bern_gen(30, 10, 0.5, 121)
y_test = []
for i in ['A','B','C']:
    y_test.extend(np.repeat(i,10))


In [3]:
def purity_score(linkage_matrix, X_test, y_test, leaf1, leaf2):
    """Compute the expected dendrogram purity.
    Sample a leaf uniformly at random. Then sample another leaf from the same
    true class uniformly at random. Find their lowest common ancestor in the
    tree and compute purity with respect to that class. (This is one of the
    evaluations used in the Bayesian Hierarchical Clustering paper).
    Args:
      root - the root with respect to which we compute purity.
    Returns:
      A float [0, 1] that represents expected dendrogram purity.
    """   
    
    N = X_test.shape[0]
    LL = [[item] for item in range(N)]
    for j in range(linkage_matrix.shape[0]):
        p, q = int(linkage_matrix[j][0]), int(linkage_matrix[j][1])
        LL.append(LL[p]+LL[q])
    common_ancestor = [item for item in LL if leaf1 in item and leaf2 in item][0]
    predict_label = np.array(y_test)[common_ancestor]
    return sum(predict_label==y_test[leaf1-1])/ len(predict_label)

In [4]:
def purity_score(linkage_matrix, y_test, class_test, repeats, seed):
    
    """Compute the expected dendrogram purity.
    Sample a leaf uniformly at random. Then sample another leaf from the same
    true class uniformly at random. Find their lowest common ancestor in the
    tree and compute purity with respect to that class. 
    return purity_score
    """   
    np.random.seed(seed)
    purity = 0
    N = len(y_test)

    for i in range(repeats):
        leaf1, leaf2 = np.random.choice(np.arange(N)[np.array(y_test)==class_test], size=2, replace=None)

        LL = [[item] for item in range(N)]
        for j in range(linkage_matrix.shape[0]):
            p, q = int(linkage_matrix[j][0]), int(linkage_matrix[j][1])
            LL.append(LL[p]+LL[q])
        common_ancestor = [item for item in LL if leaf1 in item and leaf2 in item][0]
        predict_label = np.array(y_test)[common_ancestor]

        purity += sum(predict_label==y_test[leaf1]) / len(predict_label)
    
    
    return purity / repeats

In [7]:
BHC_test = np.array(bhc.bhclust_BB(X_test)[0])
single_test = linkage(X_test,method='single')
complete_test = linkage(X_test,method='complete')
average_test = linkage(X_test,method='average')

print("BHC_test:", purity_score(BHC_test, y_test, 'A', 5, 12), purity_score(BHC_test, y_test, 'B', 5, 12), purity_score(BHC_test, y_test, 'C', 5, 12))
print("Single_linkage:", purity_score(single_test, y_test, 'A', 5, 12), purity_score(single_test, y_test, 'B', 5, 12), purity_score(single_test, y_test, 'C', 5, 12))
print("Complete_linkage:", purity_score(complete_test, y_test, 'A', 5, 12), purity_score(complete_test, y_test, 'B', 5, 12), purity_score(complete_test, y_test, 'C', 5, 12))
print("Average_linkage:", purity_score(average_test, y_test, 'A', 5, 12), purity_score(average_test, y_test, 'B', 5, 12), purity_score(average_test, y_test, 'C', 5, 12))

BHC_test: 0.961818181818 0.351973951974 0.385566137566
Single_linkage: 0.794222222222 0.309904761905 0.907692307692
Complete_linkage: 0.829120879121 0.380952380952 0.661008403361
Average_linkage: 0.755303030303 0.368421052632 0.805
